### define

In [1]:
from resources.utils import *

In [2]:
mc.list_database_names()

['admin', 'config', 'github', 'local']

In [3]:
mc['github'].list_collection_names()

['users_followers_embeddings',
 'followers',
 'users_following_embeddings',
 'followers_all',
 'users_watches',
 'repos_watches_embeddings',
 'users',
 'projects',
 'repos_stars_embeddings',
 'following']

In [4]:
!cat ./data/mysql-2019-03-01/schema.sql

SET @OLD_UNIQUE_CHECKS=@@UNIQUE_CHECKS, UNIQUE_CHECKS=0;
SET @OLD_FOREIGN_KEY_CHECKS=@@FOREIGN_KEY_CHECKS, FOREIGN_KEY_CHECKS=0;
SET @OLD_SQL_MODE=@@SQL_MODE, SQL_MODE='TRADITIONAL,ALLOW_INVALID_DATES';
SET @OLD_TIME_ZONE=@@session.time_zone;

DROP SCHEMA IF EXISTS `ghtorrent` ;
CREATE SCHEMA IF NOT EXISTS `ghtorrent` DEFAULT CHARACTER SET utf8 ;
USE `ghtorrent` ;

-- -----------------------------------------------------
-- Table `ghtorrent`.`users`
-- -----------------------------------------------------
DROP TABLE IF EXISTS `ghtorrent`.`users` ;

CREATE TABLE IF NOT EXISTS `ghtorrent`.`users` (
  `id` INT(11) NOT NULL AUTO_INCREMENT COMMENT '',
  `login` VARCHAR(255) NOT NULL COMMENT '',
  `company` VARCHAR(255) NULL DEFAULT NULL COMMENT '',
  `created_at` TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP COMMENT '',
  `type` VARCHAR(255) NOT NULL DEFAULT 'USR' COMMENT '',
  `fake` TINYINT(1) NOT NULL DEFAULT '0' COMMENT '',
  `deleted` TINYINT(1) NOT NULL DEFAULT '0' COMMENT '',
  `long`

In [5]:
def convert_dt(sdt):
    try:
        return pd.datetime.strptime(sdt, '%Y-%m-%d %H:%M:%S')
    except Exception as e:
#         print(e)
        pass

## followers all

In [6]:
followers_df = pd.read_csv('./data/mysql-2019-03-01/followers.csv',
                           header=None,
                           names=['follower_id', 'user_id', 'created_at'],
                           parse_dates=[2])
followers_df.shape

(29809738, 3)

In [7]:
followers_df.dtypes

follower_id             int64
user_id                 int64
created_at     datetime64[ns]
dtype: object

In [8]:
followers_df.sample(5)

,follower_id,user_id,created_at
623626,16486,331661,2013-06-27 03:08:46
11383304,3148103,554254,2013-09-13 22:47:49
24129648,31566290,4504583,2015-03-22 04:57:57
4500251,312720,3834473,2014-04-26 15:09:56
6634052,1137746,418425,2013-01-29 13:47:14


In [9]:
followers_df.describe()

,follower_id,user_id
count,2.980974e+07,2.980974e+07
mean,1.181476e+07,6.199693e+06
std,1.392206e+07,1.040068e+07
min,1.000000e+00,1.000000e+00
25%,1.393534e+06,1.555170e+05
50%,6.012236e+06,1.710573e+06
75%,1.333071e+07,7.163076e+06
max,4.649819e+07,4.649817e+07


In [17]:
for i in ['follower_id', 'user_id', 'created_at']:
    mc['github']['followers_all'].create_index(i)

In [33]:
to_insert = []

for uid, row in tn(followers_df.iterrows(), total=followers_df.shape[0]):
    to_insert.append(row.to_dict())
    if len(to_insert) >= 10_000:
        mc['github']['followers_all'].insert_many(to_insert)
        to_insert = []
if len(to_insert):
    mc['github']['followers_all'].insert_many(to_insert)
mc['github']['followers_all'].estimated_document_count()

29809738

## following

In [10]:
following_s = followers_df.groupby('follower_id')['user_id'].apply(list)
following_s.shape

(3833652,)

In [12]:
following_s.sample(5)

follower_id
1492978     [8150, 59801, 75710, 117467, 1221770, 5664798,...
1668790                                              [792759]
10133110                                            [1824559]
30949112                                           [36256632]
34440338                                           [32094962]
Name: user_id, dtype: object

In [13]:
following_s.map(len).describe()

count    3.833652e+06
mean     7.775807e+00
std      3.151196e+02
min      1.000000e+00
25%      1.000000e+00
50%      2.000000e+00
75%      5.000000e+00
max      2.938340e+05
Name: user_id, dtype: float64

In [14]:
for i in ['following_ids', 'following_count']:
    mc['github']['following'].create_index(i)

In [15]:
to_insert = []

for uid, fids in tn(following_s.iteritems(), total=following_s.shape[0]):
    to_insert.append({'_id': uid, 'following_ids': fids, 'following_count': len(fids)})
    if len(to_insert) >= 10_000:
        mc['github']['following'].insert_many(to_insert)
        to_insert = []
if len(to_insert):
    mc['github']['following'].insert_many(to_insert)
mc['github']['following'].estimated_document_count()

3833652

## followers

In [19]:
followers_s = followers_df.groupby('user_id')['follower_id'].apply(list)
followers_s.shape

(3793200,)

In [20]:
followers_s.sample(5)

user_id
14152966                                           [13878828]
266586      [25954, 46270, 108153, 131853, 278815, 715651,...
7151169                                    [3688872, 4712463]
2562295                                     [184862, 1998374]
21617645                                           [10095939]
Name: follower_id, dtype: object

In [21]:
followers_s.map(len).describe()

count    3.793200e+06
mean     7.858731e+00
std      1.167798e+02
min      1.000000e+00
25%      1.000000e+00
50%      2.000000e+00
75%      4.000000e+00
max      9.525800e+04
Name: follower_id, dtype: float64

In [22]:
for i in ['followers_ids', 'followers_count']:
    mc['github']['followers'].create_index(i)

In [23]:
to_insert = []

for uid, fids in tn(followers_s.iteritems(), total=followers_s.shape[0]):
    to_insert.append({'_id': uid, 'followers_ids': fids, 'followers_count': len(fids)})
    if len(to_insert) >= 10_000:
        mc['github']['followers'].insert_many(to_insert)
        to_insert = []
if len(to_insert):
    mc['github']['followers'].insert_many(to_insert)
mc['github']['followers'].estimated_document_count()

3793200

## watchers all

In [6]:
watchers_df = pd.read_csv('./data/mysql-2019-03-01/watchers.csv',
                           header=None,
                           names=['repo_id', 'user_id', 'created_at'],
                           parse_dates=[2])
watchers_df.shape

(138749808, 3)

In [7]:
watchers_df.dtypes

repo_id                int64
user_id                int64
created_at    datetime64[ns]
dtype: object

In [8]:
watchers_df.sample(5)

,repo_id,user_id,created_at
33947312,6338879,1061441,2015-08-20 06:27:26
96196833,43064597,63824,2017-04-12 12:32:26
127631316,89125366,4575050,2018-12-08 12:40:12
111082730,61491827,8744363,2018-01-28 22:23:13
34751812,6616597,675795,2016-12-03 16:50:54


In [9]:
watchers_df.describe()

,repo_id,user_id
count,1.387498e+08,1.387498e+08
mean,3.268348e+07,1.012425e+07
std,3.201465e+07,1.347603e+07
min,1.000000e+00,1.000000e+00
25%,6.602078e+06,8.030100e+05
50%,2.197623e+07,3.828345e+06
75%,5.271832e+07,1.128846e+07
max,1.276899e+08,4.973010e+07


In [11]:
for i in ['repo_id', 'user_id', 'created_at']:
    mc['github']['watchers_all'].create_index(i)

In [12]:
to_insert = []

for _, row in tn(watchers_df.iterrows(), total=watchers_df.shape[0]):
    to_insert.append(row.to_dict())
    if len(to_insert) >= 10_000:
        mc['github']['watchers_all'].insert_many(to_insert)
        to_insert = []
if len(to_insert):
    mc['github']['watchers_all'].insert_many(to_insert)
mc['github']['watchers_all'].estimated_document_count()

KeyboardInterrupt: 

## repos by watchers

In [ ]:
try:
    repos_s = watchers_df.groupby('user_id')['repo_id'].apply(list)
    m = f'[repos_s]: done {repos_s.shape[0]} users'
    tgn(m)
    print(m)
except Exception as e:
    m = f'[repos_s]: error - e'
    tgn(m)
    print(m)

In [12]:
repos_s.sample(5)

user_id
1427293     [1991708, 4066596, 9409527, 9533033, 14095875,...
2334607                      [13399, 23605, 1374453, 5641137]
3159447     [37, 1018, 1566, 3143, 7294, 12553, 21354, 277...
43837840                                 [60603847, 99422800]
43739264                                           [14359654]
Name: repo_id, dtype: object

In [13]:
repos_s.map(len).describe()

count    6.012456e+06
mean     2.307706e+01
std      1.647244e+02
min      1.000000e+00
25%      1.000000e+00
50%      2.000000e+00
75%      9.000000e+00
max      2.252900e+05
Name: repo_id, dtype: float64

In [14]:
for i in ['repos_ids', 'repos_count']:
    mc['github']['users_watches'].create_index(i)

In [15]:
to_insert = []

for uid, rids in tn(repos_s.iteritems(), total=repos_s.shape[0]):
    to_insert.append({'_id': uid, 'repos_ids': rids, 'repos_count': len(rids)})
    if len(to_insert) >= 10_000:
        mc['github']['users_watches'].insert_many(to_insert)
        to_insert = []
if len(to_insert):
    mc['github']['users_watches'].insert_many(to_insert)
mc['github']['users_watches'].estimated_document_count()

6012456

## users

In [24]:
def convert_dt(sdt):
    try:
        return pd.datetime.strptime(sdt, '%Y-%m-%d %H:%M:%S')
    except Exception as e:
        print(e)

In [25]:
users_df = pd.read_csv('./data/mysql-2019-03-01/users.csv',
                       header=None,
                       names=['id', 'login', 'company', 'created_at', 'type', 'fake', 'deleted', 'long', 'lat',
                              'country_code', 'state', 'city', 'location'],
#                        parse_dates=[3],
#                        quoting=1,
#                        quotechar='"',
#                        encoding='utf-8',
                       low_memory=False,
                       error_bad_lines=False).replace({'\\N': None}).set_index('id')
print(users_df.shape)
users_df['created_at'] = users_df['created_at'].map(convert_dt)
users_df = users_df[~users_df['created_at'].isna()]
users_df['fake'] = users_df['fake'].replace({1: True, 0: False, '1': True, '0': False}).astype(bool)
users_df['deleted'] = users_df['deleted'].replace({1: True, 0: False, '1': True, '0': False}).astype(bool)
users_df['long'] = users_df['long'].astype(float)
users_df['lat'] = users_df['lat'].astype(float)
users_df.shape

(30600304, 12)
time data ',2012-05-01 17:04:57"' does not match format '%Y-%m-%d %H:%M:%S'
time data '  OKFN Russia"' does not match format '%Y-%m-%d %H:%M:%S'
time data ' Joomla! VEL Team"' does not match format '%Y-%m-%d %H:%M:%S'
time data ' \\"Slave Center\\""' does not match format '%Y-%m-%d %H:%M:%S'
time data ' Крыша' does not match format '%Y-%m-%d %H:%M:%S'
time data ' Inc."' does not match format '%Y-%m-%d %H:%M:%S'
time data ' Microsoft Corp."' does not match format '%Y-%m-%d %H:%M:%S'
time data ' Belgrade' does not match format '%Y-%m-%d %H:%M:%S'
time data 'USR' does not match format '%Y-%m-%d %H:%M:%S'
time data ' the Real Programmers are the ones sitting in the corner  talking  about operating systems security and how to get round it.\\" [http://www.neurophys.wisc.edu/comp/humor1.html]"' does not match format '%Y-%m-%d %H:%M:%S'
time data ' I am not THAT interesting. Or is github trying to trick me into releasing personal information ?"' does not match format '%Y-%m-%d %

(30600249, 12)

In [26]:
users_df.dtypes

login                   object
company                 object
created_at      datetime64[ns]
type                    object
fake                      bool
deleted                   bool
long                   float64
lat                    float64
country_code            object
state                   object
city                    object
location                object
dtype: object

In [27]:
users_df.sample(5)

,login,company,created_at,type,fake,deleted,long,lat,country_code,state,city,location
id,,,,,,,,,,,,
5863168,AukseJ,None,2014-10-24 07:39:03,USR,False,False,0.0000,0.000000,None,None,None,None
923394,Suoshi,None,2012-06-17 07:26:28,USR,False,False,0.0000,0.000000,None,None,None,None
1019432,SDTFMWTY,None,2012-11-21 12:33:54,USR,True,False,NaN,NaN,None,None,None,None
1333349,Eurovator,None,2011-06-30 06:13:42,ORG,False,False,18.0326,59.352501,se,Stockholm County,Stockholm,Stockholm
42300826,RFSMAXBP,None,2018-03-22 13:43:54,USR,True,False,NaN,NaN,None,None,None,None


In [28]:
users_df.describe()

,long,lat
count,1.715202e+07,1.715202e+07
mean,-5.783850e-01,4.722817e+00
std,3.061493e+01,1.429124e+01
min,-1.799886e+02,-8.500000e+01
25%,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00
max,1.793745e+02,8.500000e+01


In [29]:
mc['github']['users'].create_index('login')

'login_1'

In [30]:
to_insert = []

for uid, row in tn(users_df.iterrows(), total=users_df.shape[0]):
    to_insert.append({'_id': uid, **row.to_dict()})
    if len(to_insert) >= 10_000:
        mc['github']['users'].insert_many(to_insert)
        to_insert = []
if len(to_insert):
    mc['github']['users'].insert_many(to_insert)
mc['github']['users'].estimated_document_count()

30600249

## repos topics

### research

In [1]:
!tail ./data/mysql-2019-03-01/projects.csv

127690049,"https://api.github.com/repos/edikryger/test",49702515,"test","test1",\N,"2019-02-27 09:42:55",\N,0,"2019-02-27 09:48:03",\N
127690050,"https://api.github.com/repos/ReznichenkoAlex/test-repo",49730112,"test-repo",\N,\N,"2019-02-27 09:42:52",\N,0,"2019-02-27 09:48:03",\N
127690051,"https://api.github.com/repos/Aneliukee/Floating-Islands",49730093,"Floating-Islands","Multimediju semestro projektas: kompiuterinis žaidimas Floating Islands",\N,"2019-02-27 09:42:51",127688136,0,"2019-02-27 09:48:03",1319296608
127690052,"https://api.github.com/repos/TendhyAriyatama/LD01",49730113,"LD01","Tugas PS",\N,"2019-02-27 09:42:49",\N,0,"2019-02-27 09:48:04",\N
127690053,"https://api.github.com/repos/Cat9Yuko/Cat9YuKoPixiv",47182147,"Cat9YuKoPixiv",\N,\N,"2019-02-27 09:42:48",\N,0,"2019-02-27 09:48:05",\N
127690054,"https://api.github.com/repos/blancabgz/ProyectoAngular_BlancaAbril",38922404,"ProyectoAngular_BlancaAbril","Proyecto Angular",\N,"2019-02-27 09:42:51",\N,0,"2019-02-27 09:48:05"

In [9]:
with open('./data/mysql-2019-03-01/projects.csv') as f:
    for l in tn(f):
        row = l.split(',')
        if row[0] == '77435575':
            print(l)
            break

77435575,"https://api.github.com/repos/Lamzin/math-modeling",7486329,"math-modeling",\N,\N,"2017-10-15 13:25:33",\N,0,"1970-01-02 00:00:00",\N



In [ ]:
f.close()

In [50]:
repos_df = pd.read_csv('./data/mysql-2019-03-01/projects.csv',
                       skiprows=0,
                       nrows=1_000_000,
                       header=None,
                       na_values=['\\N'],
                       names=["id", "url", "owner_id", "name", "description", "language",
                              "created_at", "forked_from", "deleted", "updated_at", "?"],
                       error_bad_lines=False,
#                        warn_bad_lines=False,
                       low_memory=False,
                      ).set_index('id')
print(repos_df.shape)
repos_df['created_at'] = repos_df['created_at'].map(convert_dt)
repos_df = repos_df[~repos_df['created_at'].isna()]
repos_df['updated_at'] = repos_df['updated_at'].map(convert_dt)
repos_df = repos_df[~repos_df['updated_at'].isna()]
repos_df.index = repos_df.index.astype(int)
repos_df['url'] = repos_df['url'].astype(str).map(lambda x: x[29:])
repos_df['deleted'] = repos_df['deleted'].replace({1: True, 0: False, 1: True, 0: False}).astype(bool)
repos_df['owner_id'] = repos_df['owner_id'].astype(int)
repos_df['forked_from'] = repos_df['forked_from'].astype(float)
repos_df['?'] = repos_df['?'].astype(float)
repos_df.shape

(1000000, 10)


(998456, 10)

In [51]:
repos_df.dtypes

url                    object
owner_id                int64
name                   object
description            object
language               object
created_at     datetime64[ns]
forked_from           float64
deleted                object
updated_at     datetime64[ns]
?                      object
dtype: object

In [52]:
repos_df.sample(5)

,url,owner_id,name,description,language,created_at,forked_from,deleted,updated_at,?
id,,,,,,,,,,
65188,okertanov/functional,35717,functional,My Functional samples & code snippets,C,2011-03-04 20:29:41,NaN,0,2016-10-13 08:42:12,NaN
349454,egw/tweepy,176567,tweepy,A Python library for accessing the Twitter API,Python,2012-05-01 02:35:25,6205.0,0,2016-10-19 10:13:51,544357
497394,travis4all/f_underscore,582667,f_underscore,A library of functions that create and compose...,JavaScript,2012-08-23 11:27:14,497392.0,0,2016-11-25 18:49:43,7363091
730327,Obijuan/obiscad,542977,obiscad,Obijuan openscad tools,OpenSCAD,2012-09-07 10:36:24,NaN,0,2019-02-25 19:34:28,NaN
816346,wetstevs/playGit,42023,playGit,A playground for tinkering with Git features.,NaN,2012-09-13 19:40:09,NaN,0,2016-11-26 17:52:36,NaN


In [35]:
repos_df.describe()

,owner_id,forked_from
count,998456.000000,4.807500e+05
mean,283153.938008,2.150832e+06
std,264088.724772,6.738677e+06
min,-1.000000,2.000000e+00
25%,61324.750000,1.699700e+04
50%,184924.500000,1.285650e+05
75%,459355.000000,7.142580e+05
max,908869.000000,5.174382e+07


In [57]:
for i in ['url', 'owner_id', 'name']:
    mc['github']['projects'].create_index(i)

In [30]:
to_insert = []

for rid, row in tn(repos_df.iterrows(), total=repos_df.shape[0]):
    to_insert.append({'_id': rid, **row.to_dict()})
    if len(to_insert) >= 10_000:
        mc['github']['projects'].insert_many(to_insert)
        to_insert = []
if len(to_insert):
    mc['github']['project'].insert_many(to_insert)
mc['github']['projects'].estimated_document_count()

30600249

### iters store

In [8]:
try:
    stop = False
    skip_count = 0
    for i in range(1000):
        print(i)
        repos_df = pd.read_csv('./data/mysql-2019-03-01/projects.csv',
                               skiprows=10_000_000*i,
                               nrows=10_000_000,
                               header=None,
                               na_values=['\\N'],
                               names=["id", "url", "owner_id", "name", "description", "language",
                                      "created_at", "forked_from", "deleted", "updated_at", "?"],
                               error_bad_lines=False,
                               low_memory=False,
                              ).set_index('id')
        c = repos_df.shape[0]
        if c < 10_000_000:
            stop = True
        repos_df['created_at'] = repos_df['created_at'].map(convert_dt)
        repos_df = repos_df[~repos_df['created_at'].isna()]
        repos_df['updated_at'] = repos_df['updated_at'].map(convert_dt)
        repos_df = repos_df[~repos_df['updated_at'].isna()]
        skip_count += (c - repos_df.shape[0])
        repos_df.index = repos_df.index.astype(int)
        repos_df['url'] = repos_df['url'].astype(str).map(lambda x: x[29:])
        repos_df['deleted'] = repos_df['deleted'].replace({1: True, 0: False, 1: True, 0: False}).astype(bool)
        repos_df['owner_id'] = repos_df['owner_id'].astype(int)
        repos_df['forked_from'] = repos_df['forked_from'].astype(float)
        repos_df['?'] = repos_df['?'].astype(float)

        to_insert = []

        for rid, row in tn(repos_df.iterrows(), total=repos_df.shape[0]):
            to_insert.append({'_id': rid, **row.to_dict()})
            if len(to_insert) >= 10_000:
                mc['github']['projects'].insert_many(to_insert)
                to_insert = []
        if len(to_insert):
            mc['github']['project'].insert_many(to_insert)
        if stop:
            break
    m = f"[projects]: stored {mc['github']['projects'].estimated_document_count()} projects, {skip_count} skiped"
    tgn(m)
except Exception as e:
    m = f'[projects]: error - {e}'
    tgn(m)
m

3



4



5



6



7



8



9



10



11


'[projects]: stored 116010000 projects, 135185 skiped'

`0 - 13412
1+2 - 19802
3 - 11092
4 - 12030
5 - 11353
6 - 11111
7 - 12384
8 - 13026
9 - 12049
10 - 11390
11 - 7536 (from 6218022 - 6210486)
Total 135185 skiped rows`

## sandbox

In [ ]:
uname = 'codebox'

udf = users_df[users_df['login'] == uname]
uid = udf.index[0]
udf

In [48]:
following_s[uid]

,follower_id,user_id,created_at


In [47]:
followers_s[uid]

,follower_id,user_id,created_at
